In [ ]:
from notebooks import *
print_sys_info()

In [ ]:
# Load model
#   - Chosen via notebooks/20180629_eval.ipynb (local runs) -> Train/test scores
search = Search.load_v0(

    # 'eval-na-dan170',
    #     'split_i=0,train=2693,test=674,classes=170', 'n_species=170,n_recs=1.0',
    #         'cls=logreg_ovr,solver=liblinear,C=0.1,class_weight=balanced',  # Not std
    #         'cls=std-sgdlog,alpha=0.1,class_weight=balanced',  # std [Re-run locally to get an n_species=170]

    'comp-l1-l2-na-ca',

        # 'split_i=0,train=28164,test=7042,classes=331', 'n_species=9,n_recs=1.0',
        #     'cls=logreg_ovr,solver=liblinear,C=1,class_weight=balanced',
        #     'cls=logreg_ovr_l1,solver=liblinear,C=1,class_weight=balanced',
        #     'cls=ovr-logreg_ovr,solver=liblinear,C=1,class_weight=balanced',
        #     'cls=ovr-logreg_ovr_l1,solver=liblinear,C=1,class_weight=balanced',

        # 'split_i=0,train=28164,test=7042,classes=331', 'n_species=82,n_recs=1.0',
        #     'cls=logreg_ovr,solver=liblinear,C=0.1,class_weight=balanced',
        #     'cls=logreg_ovr_l1,solver=liblinear,C=0.3,class_weight=balanced',
        #     'cls=ovr-logreg_ovr,solver=liblinear,C=0.01,class_weight=balanced',
        #     'cls=ovr-logreg_ovr_l1,solver=liblinear,C=0.1,class_weight=balanced',

        # 'split_i=0,train=28164,test=7042,classes=331', 'n_species=331,n_recs=1.0',
        #     'cls=ovr-logreg_ovr,solver=liblinear,C=0.01,class_weight=balanced',
        #     'cls=ovr-logreg_ovr_l1,solver=liblinear,C=0.001,class_weight=balanced',

        'split_i=0,train=34875,test=331,classes=331', 'n_species=331,n_recs=1.0',
            'cls=ovr-logreg_ovr,solver=liblinear,C=0.001,class_weight=balanced',

).fix_missing_projection_skm(
    'peterson-v0-26bae1c',
)
projection = search.projection
features = projection.features
load = features.load
display(
    search,
    search.classifier_,
    len(search.classifier_.classes_),
)

# Load user recs

In [ ]:
user_recs = load_app_recs(projection,
    # n=10,
)
display(
    df_summary(user_recs).T,
    df_value_counts(user_recs, dropna=False),
    user_recs[:10],
)

# Load xc recs

In [ ]:
# Load xc_recs
#   1. countries: Filter recs to these countries
#   2. species: Filter recs to these species
#   3. recs_at_least: Filter species to those with at least this many recs
#   4. num_species: Sample this many of the species
#   5. num_recs: Sample this many recs per species
inf = np.inf
# countries_k, com_names_k = 'na', 'us'      # 9.4k/400k -> 1.1k/60k -> 774/53k
# countries_k, com_names_k = 'na', 'ca'      # 9.4k/400k -> 1.1k/60k -> 334/35k
# countries_k, com_names_k = 'na', 'dan170'  # 9.4k/400k -> 1.1k/60k -> 170/3.4k
# countries_k, com_names_k = 'na', 'dan4'    # 9.4k/400k -> 1.1k/60k ->   4/2.2k (BEWR, HOWR, SOSP, SPTO)
countries_k, com_names_k = 'na', 'dan5'    # 9.4k/400k -> 1.1k/60k ->   5/2.3k (BEWR, HOWR, SOSP, SPTO, PSFL)
recs_at_least, num_species, num_recs =   0, inf, inf  # US[334/35k ->1.1k/60k -> 774/53k -> 774/53k   -> 743/53k]    # All US
# recs_at_least, num_species, num_recs = 100, 100, 100  # CA[334/35k -> 127/25k -> 100/21k -> 100/10k   -> 100/10k]
# recs_at_least, num_species, num_recs =  50, 100, 100  # CA[334/35k -> 224/32k -> 100/16k -> 100/ 9.0k -> 100/ 9.0k]
# recs_at_least, num_species, num_recs =   0, 100, 100  # CA[334/35k -> 334/35k -> 100/12k -> 100/ 6.8k ->  99/ 6.8k]
# recs_at_least, num_species, num_recs =   0, 168, 100  # CA[334/34k -> 334/34k -> 168/18k -> 168/11k   -> 167/11k]
# recs_at_least, num_species, num_recs =   0, 168,  20  # CA[334/34k -> 334/35k -> 168/18k -> 168/ 3.2k -> 167/ 3.2k]
# recs_at_least, num_species, num_recs =  20, 168,  20  # CA[334/35k -> 296/35k -> 168/20k -> 168/ 3.4k -> 168/ 3.4k]
# recs_at_least, num_species, num_recs =   0, inf,  20  # dan170 for app_brainstorm_1
# recs_at_least, num_species, num_recs =  20, 50,    5  # Faster dev
xc_recs, recs_stats = load_xc_recs(projection, countries_k, com_names_k, recs_at_least, num_species, num_recs)

# Precompute thumb/micro

In [ ]:
# Add df_cells: thumb*, micro*
spto = (xc_recs
    .query("species == 'SPTO'")
    # [:10]  # Faster dev
    .pipe(df_assign_first,
        # Naming: don't clobber existing cols, else e.g. downstream can't re-thumbnail from .audio
        thumb      = df_cell_spectros(plot_thumb.many, features, thumb_s=2, scale=dict(h=13)),
        thumb2     = df_cell_spectros(plot_thumb.many, features, thumb_s=2, scale=dict(h=26)),
        thumb_full = df_cell_spectros(plot_thumb.many, features, thumb_s=2),
        micro      = df_cell_spectros(plot_spectro_micro.many, features, wrap_s=26, scale=dict(h=13)),
        micro2     = df_cell_spectros(plot_spectro_micro.many, features, wrap_s=13, scale=dict(h=26)),
        micro_full = df_cell_spectros(plot_spectro_wrap.many, features),
    )
)
spto_a = (spto
    .query("quality == 'A'")
)

# Util

# TODO Did I forget pca for all of app_ideas_5!? (And it worked decently well?)
- dist(feat) -> dist(probs) -> dist(pca(2, probs))

## TODO How do we grok all of these?
- Many result lists per query_rec (below) X many query_recs (notebooks/app_ideas_5, on left)...
- (Maybe overall simpler to defer this up-front eval and let me/Alex select on the fly in the app...? Or maybe not?)

## TODO Think harder: pca(spto_a) vs. pca(spto) vs. pca(xc_recs)?

In [ ]:
# Plot: spto_a: .4 var ~ 9 components
# plot_pca_var_explained(pca_spto_a.pca_var_, max_var_explained=.5)

In [ ]:

def user_query_rec(
    pca: recs_pca_norm,
    user_rec: Recording,
    show_rec=True,
    show_probs=10,
    search=search,
    show_spectro=True,
    **kwargs,
) -> Recording:
    if show_rec: print(rec_str_line(user_rec))
    if show_probs: display(rec_probs(user_rec, search)[:show_probs])
    if show_spectro: display(plot_spectro_micro(user_rec, features, **kwargs, show=False))  # display(show=False) for audio
    return pca.transform_rec(user_rec)

def table_recs(
    recs,
    limit=None,
    more_cols=[],
    cols=xc_eda_cols,
    decorate=True,
    dist_bar_size=30,
    handtype_cmap=mpl_cmap_concat('tab10', 'tab20'),
    color_stack=False,
    **kwargs_recs_decorate,
):
    if decorate:
        recs = recs_decorate(recs, **kwargs_recs_decorate)
    return (recs
        .join(how='left', other=load_xc_handtype())
        .pipe(df_col_map, handtype=lambda xs: [x[:3] for x in xs])
        .pipe(df_col_color_d, handtype=handtype_cmap, _stack=color_stack)
        [:limit]
        .pipe(lambda df: df if 'dist' not in df.columns else df_assign_first(df,
            dist_bar=lambda df: df.dist.map(lambda dist: text_bar(dist, max=df.dist.max(), norm=dist_bar_size, side='right')),
        ))
        .pipe(lambda df: df[[
            c
            for c in [
                'dist_bar', 'dist', 'handtype',
                'thumb', 'micro',
                *more_cols,
                *cols,
            ]
            if c in df.columns
        ]])
    )

@dataclass
class proba_pca_slice:
    components: Union[int, float]
    def __call__(self, df):
        proba_pca_var = df.proba_pca_var.iloc[0]
        return [
            row.proba_pca[:n]
            for row in df_rows(df)
            for n in [
                len(row.proba_pca_var) if self.components is None else
                self.components if isinstance(self.components, int) else
                (row.proba_pca_var <= self.components).sum()
            ]
        ]

def grid_recs(
    search_recs,
    query_rec: QueryRec,
    pca_components,
    faster_dev=False,
    limit=35,
    decorate=True,
    dist_bar_size=10,
    **kwargs,
) -> pd.DataFrame:
    if decorate:
        search_recs = sub_kwargs(recs_decorate, **kwargs)(search_recs)
    bys = [
        (lambda x: 'f', Search.X),
        (lambda x: 'p', search.species_proba),
        *[(lambda x: x.components, proba_pca_slice(c)) for c in pca_components],
    ]
    if faster_dev:
        limit = 10
        bys = bys[:3]
    return DF(dict(flatten(
        [
            ('%s/i' % by_desc, df.xc_id),
            ('%s/d' % by_desc, df_map_rows(df, lambda row: df_cell_stack([row.dist_bar, row.dist]))),
            ('%s/h' % by_desc, df.handtype),
            *([('%s/t' % by_desc, df.thumb)] if 'micro_s' not in kwargs or 'thumb_s' in kwargs else []),
            *([('%s/m' % by_desc, df.micro)] if 'micro_s' in kwargs else []),
        ]
        for by_desc_f, by in bys
        for by_desc in [by_desc_f(by)]
        for df in [
            sub_kwargs(table_recs, **kwargs)(
                recs=rec_neighbors_by(search_recs=search_recs, query_rec=query_rec, by=by),
                limit=limit, dist_bar_size=dist_bar_size,
                color_stack=True,
                decorate=False,  # Already done above (once, instead of len(bys) times here)
            )
            .sort_values('dist')
            .reset_index()
        ]
    )))

def recs_decorate(
    recs,
    size='big',  # 'big' | 'small'
    thumb_s=2,
    micro_s=13,
):
    cols = dict()
    if size == 'big':
        cols.update(dict(
            thumb=df_cell_spectros(plot_thumb.many, features, thumb_s=thumb_s, scale=dict(h=26)),
            micro=df_cell_spectros(plot_spectro_micro.many, features, wrap_s=micro_s, scale=dict(h=26)),
        ))
    else:
        cols.update(dict(
            thumb=df_cell_spectros(plot_thumb.many, features, thumb_s=thumb_s, scale=dict(h=13)),
            micro=df_cell_spectros(plot_spectro_micro.many, features, wrap_s=micro_s*2, scale=dict(h=13)),  # HACK micro_s*2
        ))
    return df_assign_first(recs, **cols)

In [ ]:
pca_spto_a = recs_pca_norm(by=search.species_proba, out_col='proba_pca').fit_recs(spto_a)
# limit = 20; pca_components = [1., .9, .8, .7, .6, .5, .4, 8, 7, 6, 5, 4, 3, 2, 1]
# limit = 10; pca_components = [1., .5, 1]
limit = 5; pca_components = [1.]

In [ ]:
grid_recs(
    query_rec=user_query_rec(pca_spto_a,
        user_recs.loc['20180429-dcf3'].pipe(projection.slice_spectro, 16.84, 17.92),  # SPTO spotted towhee song- kings rd
        show_probs=12,
    ),
    search_recs=pca_spto_a.transform_recs(spto_a),
    limit=limit, pca_components=pca_components,
    thumb_s=3,
    # micro_s=10,
)

In [ ]:
grid_recs(
    query_rec=user_query_rec(pca_spto_a,
        user_recs.loc['20160923-0ac1'].pipe(projection.slice_spectro, 12, 13.5),  # Turkey purring
        show_probs=12,
    ),
    search_recs=pca_spto_a.transform_recs(spto_a),
    limit=limit, pca_components=pca_components,
    thumb_s=3,
    # micro_s=10,
)

In [ ]:
grid_recs(
    query_rec=312024,  # coastal
    search_recs=pca_spto_a.transform_recs(spto_a),
    limit=limit, pca_components=pca_components,
    thumb_s=3,
    # micro_s=10,
)

In [ ]:
grid_recs(
    query_rec=127014,  # wheeze (noisy)
    search_recs=pca_spto_a.transform_recs(spto_a),
    limit=limit, pca_components=pca_components,
    thumb_s=3,
    # micro_s=10,
)

In [ ]:
grid_recs(
    query_rec=159823,  # wheeze
    search_recs=pca_spto_a.transform_recs(spto_a),
    limit=limit, pca_components=pca_components,
    thumb_s=3,
    # micro_s=10,
)

In [ ]:
grid_recs(
    query_rec=182566,  # wheeze
    search_recs=pca_spto_a.transform_recs(spto_a),
    limit=limit, pca_components=pca_components,
    thumb_s=3,
    # micro_s=10,
)

In [ ]:
grid_recs(
    query_rec=168046,  # mx (not many mx's)
    search_recs=pca_spto_a.transform_recs(spto_a),
    limit=limit, pca_components=pca_components,
    thumb_s=3,
    # micro_s=10,
)

In [ ]:
grid_recs(
    query_rec=274874,  # interior
    search_recs=pca_spto_a.transform_recs(spto_a),
    limit=limit, pca_components=pca_components,
    thumb_s=3,
    # micro_s=10,
)

In [ ]:
grid_recs(
    query_rec=126698,  # coastal (weird)
    search_recs=pca_spto_a.transform_recs(spto_a),
    limit=limit, pca_components=pca_components,
    thumb_s=3,
    # micro_s=10,
)